# Experiment 014: NFP-based Local Search

Implement No-Fit Polygon (NFP) algorithm for fast collision detection.
NFP enables O(1) collision checks by precomputing forbidden regions.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon, Point
from shapely import affinity
from shapely.ops import unary_union
import json
import time
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 30
SCALE_FACTOR = Decimal('1e18')

print("Setup complete")

Setup complete


In [2]:
# Tree shape vertices
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def create_tree_polygon(x, y, angle):
    x, y, angle = float(x), float(y), float(angle)
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def create_high_precision_tree(x, y, angle):
    x = Decimal(str(x))
    y = Decimal(str(y))
    angle = Decimal(str(angle))
    sf = SCALE_FACTOR
    vertices = [
        (float(Decimal('0.0') * sf), float(Decimal('0.8') * sf)),
        (float(Decimal('0.125') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('-0.125') * sf), float(Decimal('0.5') * sf)),
    ]
    poly = Polygon(vertices)
    poly = affinity.rotate(poly, float(angle), origin=(0, 0))
    poly = affinity.translate(poly, xoff=float(x * sf), yoff=float(y * sf))
    return poly

def validate_no_overlap_strict(trees_data):
    if len(trees_data) <= 1:
        return True
    polygons = [create_high_precision_tree(t['x'], t['y'], t['deg']) for t in trees_data]
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                return False
    return True

def get_bbox_side(trees):
    if len(trees) == 0:
        return 0
    polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    union = unary_union(polygons)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def get_score(trees, n):
    side = get_bbox_side(trees)
    return (side ** 2) / n

print("Core functions defined")

Core functions defined


In [3]:
# NFP (No-Fit Polygon) Implementation
# NFP of A and B is the region where B's reference point cannot be placed
# without B overlapping A

def compute_nfp_simple(poly_a, poly_b):
    """Compute NFP using Minkowski sum approach.
    NFP = poly_a (+) (-poly_b) where (+) is Minkowski sum
    """
    # Get vertices of both polygons
    coords_a = list(poly_a.exterior.coords)[:-1]  # Remove closing point
    coords_b = list(poly_b.exterior.coords)[:-1]
    
    # Reflect poly_b around origin
    coords_b_reflected = [(-x, -y) for x, y in coords_b]
    
    # Minkowski sum: for each vertex of A, add all vertices of reflected B
    nfp_points = []
    for ax, ay in coords_a:
        for bx, by in coords_b_reflected:
            nfp_points.append((ax + bx, ay + by))
    
    # Create convex hull of all points (approximation for non-convex polygons)
    from shapely.geometry import MultiPoint
    nfp = MultiPoint(nfp_points).convex_hull
    
    return nfp

def precompute_nfps(angles=[0, 45, 90, 135, 180, 225, 270, 315]):
    """Precompute NFPs for common angle pairs."""
    nfp_cache = {}
    base_tree = Polygon(list(zip(TX, TY)))
    
    for angle_a in angles:
        for angle_b in angles:
            tree_a = affinity.rotate(base_tree, angle_a, origin=(0, 0))
            tree_b = affinity.rotate(base_tree, angle_b, origin=(0, 0))
            nfp = compute_nfp_simple(tree_a, tree_b)
            nfp_cache[(angle_a, angle_b)] = nfp
    
    return nfp_cache

print("NFP functions defined")

# Precompute NFPs
print("Precomputing NFPs...")
nfp_cache = precompute_nfps()
print(f"Precomputed {len(nfp_cache)} NFPs")

NFP functions defined
Precomputing NFPs...
Precomputed 64 NFPs


In [4]:
def check_overlap_nfp(tree_new, trees_existing, nfp_cache):
    """Check if tree_new overlaps with any existing tree using NFP.
    Returns True if there's an overlap.
    """
    x_new, y_new = float(tree_new['x']), float(tree_new['y'])
    angle_new = float(tree_new['deg'])
    
    # Quantize angle to nearest cached angle
    cached_angles = [0, 45, 90, 135, 180, 225, 270, 315]
    angle_new_q = min(cached_angles, key=lambda a: abs((a - angle_new) % 360))
    
    for tree_exist in trees_existing:
        x_exist, y_exist = float(tree_exist['x']), float(tree_exist['y'])
        angle_exist = float(tree_exist['deg'])
        angle_exist_q = min(cached_angles, key=lambda a: abs((a - angle_exist) % 360))
        
        # Get NFP for this angle pair
        nfp = nfp_cache.get((angle_exist_q, angle_new_q))
        if nfp is None:
            continue
        
        # Translate NFP to existing tree's position
        nfp_translated = affinity.translate(nfp, x_exist, y_exist)
        
        # Check if new tree's position is inside NFP (means overlap)
        point = Point(x_new, y_new)
        if nfp_translated.contains(point):
            return True
    
    return False

print("NFP overlap check function defined")

NFP overlap check function defined


In [5]:
# Load current best submission
print("Loading current best submission...")
df = pd.read_csv('/home/code/experiments/012_full_snapshot_ensemble/submission.csv')
df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

best_trees = {}
best_scores = {}

for n, g in df.groupby('N'):
    trees = []
    for _, row in g.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append({'x': x, 'y': y, 'deg': deg})
    best_trees[n] = trees
    best_scores[n] = get_score(trees, n)

baseline_total = sum(best_scores.values())
print(f"Baseline score: {baseline_total:.6f}")

Loading current best submission...


Baseline score: 70.340917


In [6]:
def nfp_local_search(trees, n, max_iter=100, step_sizes=[0.01, 0.005, 0.002, 0.001]):
    """Local search using NFP for fast collision detection.
    Try moving each tree in small steps to reduce bounding box.
    """
    current_trees = [dict(t) for t in trees]
    current_score = get_score(current_trees, n)
    best_score = current_score
    best_trees = [dict(t) for t in current_trees]
    
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    
    improved = True
    iteration = 0
    
    while improved and iteration < max_iter:
        improved = False
        iteration += 1
        
        for i in range(len(current_trees)):
            for step in step_sizes:
                for dx, dy in directions:
                    # Try moving tree i
                    test_trees = [dict(t) for t in current_trees]
                    test_trees[i]['x'] = str(float(test_trees[i]['x']) + dx * step)
                    test_trees[i]['y'] = str(float(test_trees[i]['y']) + dy * step)
                    
                    # Quick NFP check first
                    other_trees = test_trees[:i] + test_trees[i+1:]
                    if check_overlap_nfp(test_trees[i], other_trees, nfp_cache):
                        continue
                    
                    # If NFP says OK, do strict validation
                    if not validate_no_overlap_strict(test_trees):
                        continue
                    
                    # Calculate new score
                    new_score = get_score(test_trees, n)
                    
                    if new_score < best_score - 1e-9:
                        best_score = new_score
                        best_trees = [dict(t) for t in test_trees]
                        current_trees = [dict(t) for t in test_trees]
                        improved = True
    
    return best_trees, best_score, best_score < current_score - 1e-9

print("NFP local search function defined")

NFP local search function defined


In [ ]:
# Test on a few N values
print("\nTesting NFP local search on N=50, 100, 150...")

for test_n in [50, 100, 150]:
    trees = best_trees[test_n]
    old_score = best_scores[test_n]
    
    start = time.time()
    new_trees, new_score, improved = nfp_local_search(trees, test_n, max_iter=20)
    elapsed = time.time() - start
    
    if improved:
        print(f"  N={test_n}: {old_score:.6f} -> {new_score:.6f} (+{old_score - new_score:.6f}) in {elapsed:.1f}s")
    else:
        print(f"  N={test_n}: No improvement found in {elapsed:.1f}s")

In [ ]:
# Apply to all N values with improvements tracking
print("\n" + "=" * 60)
print("APPLYING NFP LOCAL SEARCH TO ALL N VALUES")
print("=" * 60)

improvements = []
start_time = time.time()

for n in range(2, 201):  # Skip N=1 (already optimal)
    trees = best_trees[n]
    old_score = best_scores[n]
    
    # Use fewer iterations for larger N
    max_iter = 30 if n <= 50 else 20 if n <= 100 else 10
    
    new_trees, new_score, improved = nfp_local_search(trees, n, max_iter=max_iter)
    
    if improved:
        best_trees[n] = new_trees
        best_scores[n] = new_score
        improvement = old_score - new_score
        improvements.append((n, improvement))
        print(f"  N={n:3d}: +{improvement:.6f}")
    
    if n % 50 == 0:
        elapsed = time.time() - start_time
        print(f"  Progress: N={n}/200, elapsed={elapsed:.1f}s, improvements={len(improvements)}")

print(f"\nTotal time: {time.time() - start_time:.1f}s")
print(f"N values improved: {len(improvements)}")
if improvements:
    total_improvement = sum(imp for _, imp in improvements)
    print(f"Total improvement: {total_improvement:.6f}")

In [ ]:
# Calculate final score
final_total = sum(best_scores.values())
print(f"\n" + "=" * 60)
print("RESULTS")
print("=" * 60)
print(f"Baseline score: {baseline_total:.6f}")
print(f"Final score: {final_total:.6f}")
print(f"Improvement: {baseline_total - final_total:.6f}")
print(f"Target: 68.879235")
print(f"Gap to target: {final_total - 68.879235:.6f}")

In [ ]:
# Final validation
print("\n" + "=" * 60)
print("FINAL VALIDATION")
print("=" * 60)

final_overlaps = []
for n in range(1, 201):
    if not validate_no_overlap_strict(best_trees[n]):
        final_overlaps.append(n)

if final_overlaps:
    print(f"WARNING: {len(final_overlaps)} N values have overlaps!")
else:
    print("All N values pass strict validation!")

In [ ]:
# Create submission
print("\n" + "=" * 60)
print("CREATE SUBMISSION")
print("=" * 60)

rows = []
for n in range(1, 201):
    trees = best_trees[n]
    for i, t in enumerate(trees):
        x_val = str(t['x']).replace('s', '')
        y_val = str(t['y']).replace('s', '')
        deg_val = str(t['deg']).replace('s', '')
        rows.append({
            'id': f"{n:03d}_{i}",
            'x': f"s{x_val}",
            'y': f"s{y_val}",
            'deg': f"s{deg_val}"
        })

submission_df = pd.DataFrame(rows)
print(f"Submission shape: {submission_df.shape}")

submission_df.to_csv('/home/code/experiments/014_nfp_local_search/submission.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print("Submission saved!")

In [ ]:
# Save metrics
metrics = {
    'cv_score': final_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - final_total,
    'n_improved': len(improvements),
    'final_overlaps': len(final_overlaps),
    'target': 68.879235,
    'gap': final_total - 68.879235
}

with open('/home/code/experiments/014_nfp_local_search/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\nMetrics saved!")
print(json.dumps(metrics, indent=2))